## Classe Decision Tree

In [ ]:
import numpy as np
import math

def std_agg(cnt, s1, s2): return math.sqrt((s2/cnt) - (s1/cnt)**2)

class DecisionTree():
    def __init__(self, x, y, idxs=None, min_leaf=2):
        if idxs is None: idxs = np.arange(len(y))
        self.x, self.y, self.idxs, self.min_leaf = x, y, idxs, min_leaf
        self.n, self.c = len(idxs), x.shape[1]
        self.val = np.mean(y[idxs])
        self.score = float('inf')
        self.find_varsplit()

    def find_varsplit(self):
        for i in range(self.c): self.find_better_split(i)
        if self.score == float('inf'): return
        x = self.split_col
        lhs = np.nonzero(x <= self.split)[0]
        rhs = np.nonzero(x > self.split)[0]
        self.lhs = DecisionTree(self.x, self.y, self.idxs[lhs])
        self.rhs = DecisionTree(self.x, self.y, self.idxs[rhs])

    def find_better_split(self, var_idx):
        x, y = self.x.values[self.idxs, var_idx], self.y[self.idxs]
        sort_idx = np.argsort(x)
        sort_y, sort_x = y[sort_idx], x[sort_idx]
        rhs_cnt, rhs_sum, rhs_sum2 = self.n, sort_y.sum(), (sort_y ** 2).sum()
        lhs_cnt, lhs_sum, lhs_sum2 = 0, 0., 0.

        for i in range(0, self.n - self.min_leaf - 1):
            xi, yi = sort_x[i], sort_y[i]
            lhs_cnt += 1;
            rhs_cnt -= 1
            lhs_sum += yi;
            rhs_sum -= yi
            lhs_sum2 += yi ** 2;
            rhs_sum2 -= yi ** 2
            if i < self.min_leaf or xi == sort_x[i + 1]:
                continue

            lhs_std = std_agg(lhs_cnt, lhs_sum, lhs_sum2)
            rhs_std = std_agg(rhs_cnt, rhs_sum, rhs_sum2)
            curr_score = lhs_std * lhs_cnt + rhs_std * rhs_cnt
            if curr_score < self.score:
                self.var_idx, self.score, self.split = var_idx, curr_score, xi

    @property
    def split_name(self):
        return self.x.columns[self.var_idx]

    @property
    def split_col(self):
        return self.x.values[self.idxs, self.var_idx]

    @property
    def is_leaf(self):
        return self.score == float('inf')

    def __repr__(self):
        s = f'n: {self.n}; val:{self.val}'
        if not self.is_leaf:
            s += f'; score:{self.score}; split:{self.split}; var:{self.split_name}'
        return s

    def predict(self, x):
        return np.array([self.predict_row(xi) for xi in x])

    def predict_row(self, xi):
        if self.is_leaf: return self.val
        t = self.lhs if xi[self.var_idx] <= self.split else self.rhs
        return t.predict_row(xi)

# Imports

In [ ]:
%matplotlib inline

import pandas as pd
import numpy as np
from IPython.display import display
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import shap
import matplotlib.pylab as pl

# Gradient Boosting

## Criando dataset

In [ ]:
x = np.arange(0,50)
x = pd.DataFrame({'x':x})

y1 = np.random.uniform(10,15,10)
y2 = np.random.uniform(20,25,10)
y3 = np.random.uniform(0,5,10)
y4 = np.random.uniform(30,32,10)
y5 = np.random.uniform(13,17,10)

y = np.concatenate((y1,y2,y3,y4,y5))
y = y[:,None]

In [ ]:
x.shape, y.shape

In [ ]:
plt.figure(figsize=(15,7))
plt.plot(x,y, 'o')
plt.title("x vs. y")
plt.xlabel("x")
plt.ylabel("y")
plt.show()

## Aprendizagem Fraca

A nossa árvore de decisão encontra o melhor jeito de dividir o dataset em dois grupos, usando o método `find_better_split`

In [ ]:
predf = 0

tree = DecisionTree(x,y)
tree.find_better_split(0)
r = np.where(x == tree.split)[0][0]


f, ax = plt.subplots(sharey=True, figsize = (13,5))
ax.plot(x, y, 'o')
ax.set_title(f'Divisão da Árvore de Decisão')
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.axvline(r, c="k", ls="--")

Para cada um dos dois grupos o nosso _weak learner_ prevê encontra a média dos elementos. O processo pode ser repetido, a partir dos residuos desse modelo

In [ ]:
n = len(y)

left_idx = np.where(x <= tree.split)[0]
right_idx = np.where(x > tree.split)[0]

predi = np.zeros(n)
np.put(predi, left_idx, np.repeat(np.mean(y[left_idx]), r))
np.put(predi, right_idx, np.repeat(np.mean(y[right_idx]), n-r))

predi = predi[:,None]

f, (ax1, ax2) = plt.subplots(1, 2, sharey=True, figsize = (20,5))
ax1.plot(x, y, 'o')
ax1.plot(x, predi, c="k", ls="--")
ax1.set_title(f'Weak Learner')
ax1.set_xlabel('x')
ax1.set_ylabel('y')
ax1.axvline(r, c="k", ls="--")


tree = DecisionTree(x,y - predi)
tree.find_better_split(0)
r = np.where(x == tree.split)[0][0]

ax2.plot(x, y - predi, 'o')
ax2.set_title(f'Residuos (repete-se o processo)')
ax2.set_xlabel('x')
ax2.set_ylabel('Residuo')
ax2.axvline(r, c="k", ls="--")

In [ ]:
xi = x
yi = y
ei = 0
n = len(yi)
predf = 0

for i in range(2):
    tree = DecisionTree(xi,yi)
    tree.find_better_split(0)
    
    r = np.where(xi == tree.split)[0][0]    
    
    left_idx = np.where(xi <= tree.split)[0]
    right_idx = np.where(xi > tree.split)[0]
    
    predi = np.zeros(n)
    np.put(predi, left_idx, np.repeat(np.mean(yi[left_idx]), r))
    np.put(predi, right_idx, np.repeat(np.mean(yi[right_idx]), n-r))
    
    predi = predi[:,None]
    predf = predf + predi
    
    ei = y - predf
    yi = ei
    
    
    xa = np.array(x.x) 
    order = np.argsort(xa)
    xs = np.array(xa)[order]
    ys = np.array(predf)[order]

    f, (ax1, ax2, ax3) = plt.subplots(1, 3, sharey=True, figsize = (18,3.5))

    ax1.plot(x, y - (predf - predi), 'go')
    ax1.plot(x, predi, 'g', ls="--")
    ax1.set_title(f'Weak Learner (Iteração {i+1})')
    ax1.set_xlabel('x')
    ax1.set_ylabel('yi')
    ax1.axvline(r, c="k", ls="--")
    
    ax2.plot(x,y, 'o')
    ax2.plot(xs, ys, 'r')
    ax2.set_title(f'Previsão Final (Iteração {i+1})')
    ax2.set_xlabel('x')
    ax2.set_ylabel('y')

    ax3.plot(x, ei, 'go')
    ax3.set_title(f'Residuos (Iteração {i+1})')
    ax3.set_xlabel('x')
    ax3.set_ylabel('Residuals')
    
    

# XGBoost

### Dataset Boston

Dataset com preços de imóveis em Boston


#### Atributos
- **CRIM** - Crime per capita
- **ZN** - Proporção de área residencial
- **INDUS** - Proporção de área industrial
- **CHAS** - Charles River dummy variable
- **NOX** -  Concentração de óxido nítrico (partes por 10 milhões)
- **RM** - Média de cômodos por habitação
- **AGE** - Proporção de casas/prédios construídos antes de 1940
- **DIS** - Distância (ponderada) para centros de emprego
- **RAD** - Índice de acessibilidade a vias centrais
- **TAX** - Taxa de imposto
- **PTRATIO** - Proporção número de professores/número de estudantes
- **B** - 1000(Bk — 0.63)², onde Bk é a proporção de negros
- **LSTAT** - Status da população

#### Target
- **MEDV** - Mediana do valor dos imóveis

In [ ]:
boston = load_boston()
X = pd.DataFrame(boston.data, columns=boston.feature_names)
y = pd.Series(boston.target)

In [ ]:
X.head()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 20)

In [ ]:
regressor = xgb.XGBRegressor(
    n_estimators=100,
    reg_lambda=1,
    gamma=0,
    reg_alpha=0,
    max_depth=3,
    min_child_weight=1,
    random_state=42
)

In [ ]:
regressor.fit(X_train, y_train)

In [ ]:
pd.DataFrame(regressor.feature_importances_.reshape(1, -1), columns=boston.feature_names)

In [ ]:
print(f"EQM (Treino): {mean_squared_error(y_train, regressor.predict(X_train))}")

In [ ]:
print(f"EQM (Teste): {mean_squared_error(y_test, regressor.predict(X_test))}")

## Hyperparâmetros

### Max Depth

In [ ]:
regressor = xgb.XGBRegressor(
    n_estimators=100,
    reg_lambda=1,
    gamma=0,
    reg_alpha=0,
    max_depth=5,
    min_child_weight=1,
    random_state=42
)
regressor.fit(X_train, y_train)
print(f"MSE (Treino): {mean_squared_error(y_train, regressor.predict(X_train))}")

In [ ]:
print(f"MSE (Teste): {mean_squared_error(y_test, regressor.predict(X_test))}")

### Min Child Weight

In [ ]:
regressor = xgb.XGBRegressor(
    n_estimators=100,
    reg_lambda=1,
    gamma=0,
    reg_alpha=0,
    max_depth=5,
    random_state=42,
    min_child_weight=7
)
regressor.fit(X_train, y_train)
print(f"MSE (Treino): {mean_squared_error(y_train, regressor.predict(X_train))}")

In [ ]:
print(f"MSE (Teste): {mean_squared_error(y_test, regressor.predict(X_test))}")

### Reg_Alpha

In [ ]:
regressor = xgb.XGBRegressor(
    n_estimators=100,
    reg_lambda=1,
    gamma=0,
    reg_alpha=0.1,
    max_depth=5,
    random_state=42,
    min_child_weight=7
)
regressor.fit(X_train, y_train)
print(f"MSE (Treino): {mean_squared_error(y_train, regressor.predict(X_train))}")

In [ ]:
print(f"MSE (Teste): {mean_squared_error(y_test, regressor.predict(X_test))}")

In [ ]:
regressor = xgb.XGBRegressor(
    n_estimators=100,
    reg_lambda=1,
    gamma=0,
    reg_alpha=0.001,
    max_depth=5,
    random_state=42,
    min_child_weight=7
)
regressor.fit(X_train, y_train)
print(f"MSE (Treino): {mean_squared_error(y_train, regressor.predict(X_train))}")

In [ ]:
print(f"MSE (Teste): {mean_squared_error(y_test, regressor.predict(X_test))}")

# Exercícios

### Dados de diabetes
Dados de 442 pacientes de diabetes (idade, sexo, medidas de serum, ...), alem de uma variável-resposta de progressao da doença um ano depois

#### Atributos
- **Idade**
- **Sexo**
- **IMC**
- **Pressão Sanguinea**
- **S1**
- **S2**
- **S3**
- **S4**
- **S5**
- **S6**


#### Target
Medida quantitativa de progressão da doença

In [ ]:
from sklearn.datasets import load_diabetes

In [ ]:
diabetes = load_diabetes()

X = diabetes.data
y = diabetes.target

X

# Classificação

## Iris dataset

#### Atributos
- Comprimento do sépalo
- Largura do sépalo
- Comprimento da pétala
- Largura da pétala

#### Target
- Classe: Iris Setosa, Iris Versicolour e Iris Virginica

In [ ]:
from sklearn import datasets
from sklearn import metrics

iris = datasets.load_iris()
X = iris.data
y = iris.target

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from xgboost import XGBClassifier
clf = XGBClassifier(objective = 'multi:softmax')
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

In [ ]:
print(f"Acurácia: {metrics.accuracy_score(y_test, y_pred)}")

## Early Stopping

### Adult Dataset

In [ ]:
X,y = shap.datasets.adult()

# create a train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=7)
d_train = xgb.DMatrix(X_train, label=y_train)
d_test = xgb.DMatrix(X_test, label=y_test)

In [ ]:
params = {
    "eta": 0.01,
    "objective": "binary:logistic",
    "subsample": 0.5,
    "base_score": np.mean(y_train),
    "eval_metric": "logloss"
}
model = xgb.train(params, d_train, 5000, evals = [(d_test, "test")], verbose_eval=100, early_stopping_rounds=20)

## Feature Importance

In [ ]:
xgb.plot_importance(model)
pl.title("xgboost.plot_importance(model)")
pl.show()

In [ ]:
xgb.plot_importance(model, importance_type="gain")
pl.title('xgboost.plot_importance(model, importance_type="gain")')
pl.show()

## SHAP

In [ ]:
shap.initjs()

In [ ]:
X_sample = X.sample(500)

explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_sample)

In [ ]:
shap.summary_plot(shap_values, X_sample, plot_type="bar")

In [ ]:
shap.force_plot(explainer.expected_value, shap_values[0,:], X_sample.iloc[0,:])

In [ ]:
shap.summary_plot(shap_values, X_sample)